### Prompt:
You are an analyst for a fastfood restaurant. The company has tasked you to analyze their past transactions. Each row represents a single transaction/order from one of their ten (10) branches. The order is recorded as a __`string`__ which is formatted as: 

"`[quantity1][item_code1], [quantity2][item_code2], [quantity3][item_code3]`"

However, because of some technical error, the __number of combo meals purchased__ as well as the __total order value__ was lost, so we'll need to restore that.

### You are tasked to come up with the following:
1. Given the order column, count how many of each product was purchased
2. Given each product count, get the number of supposed combo meals purchased. <br><i>(You may update the product count when the combo meals are counted)</i>.
3. Calculate the total order value.
4. Determine which branch made the most revenue, and how much?

### The combo meals are detailed below:
1. Family combo: 4 burgers, 4 fries, 4 drinks
2. Big combo: 1 burgers, 1 fries, 1 drinks
3. Snack combo: 1 fries, 1 drinks

In [28]:
import pandas as pd

df = pd.read_csv('food_transactions.csv')
df

,Unnamed: 0,date,branch,order
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink"
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink"
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink"
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink"
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink"
...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink"
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries"
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries"
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink"


In [2]:
pricedf = pd.read_json('price_list.json')
pricedf

,price
Big_combo,169
Fam_combo,599
S_Burger,89
S_Drink,49
S_Fries,69
Snack_combo,99


In [29]:
# 1. Given the order column, count how many of each product was purchased
def find_burger(text):
    text = str(text)
    if 'S_Burger' in text:
        num = text.find('S_Burger')
        return text[num-1]
    else:
        return 0

def find_fries(text):
    text = str(text)
    if 'S_Fries' in text:
        num = text.find('S_Fries')
        return text[num-1]
    else:
        return 0

def find_drink(text):
    text = str(text)
    if 'S_Drink' in text:
        num = text.find('S_Drink')
        return text[num-1]
    else:
        return 0


df['burger'] = df['order'].apply(find_burger)
df['fries'] = df['order'].apply(find_fries)
df['drink'] = df['order'].apply(find_drink)
df

,Unnamed: 0,date,branch,order,burger,fries,drink
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink",1,4,5
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink",2,4,3
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink",2,4,2
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink",0,6,1
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink",5,0,6
...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink",0,3,6
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries",7,6,0
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries",4,6,0
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink",7,6,5


In [30]:
# 2. Given each product count, get the number of supposed combo meals purchased.
# (You may update the product count when the combo meals are counted).
# The combo meals are detailed below:
# Family combo: 4 burgers, 4 fries, 4 drinks
# Big combo: 1 burgers, 1 fries, 1 drinks
# Snack combo: 1 fries, 1 drinks

def product_count(row):
    burger_num = int(row['burger']) // 4
    fries_num = int(row['fries']) // 4 
    drink_num = int(row['drink']) // 4 

    if burger_num > 0 and fries_num > 0 and drink_num > 0:
        return min(burger_num, fries_num, drink_num)
    else:
        return 0
    



df['family_combo'] = df.apply(product_count, axis = 1)

df['burger'] = df.apply(lambda x: int(x.burger) - (x.family_combo * 4), axis = 1)

df['fries'] = df.apply(lambda x: int(x.fries) - (x.family_combo * 4), axis = 1)

df['drink'] = df.apply(lambda x: int(x.drink) - (x.family_combo * 4), axis = 1)

'================================================'

def big_count(row):
    burger_num = int(row['burger']) // 1
    fries_num = int(row['fries']) // 1 
    drink_num = int(row['drink']) // 1 

    if burger_num > 0 and fries_num > 0 and drink_num > 0:
        return min(burger_num, fries_num, drink_num)
    else:
        return 0


df['big_combo'] = df.apply(big_count, axis = 1)

df['burger'] = df.apply(lambda x: int(x.burger) - (x.big_combo * 1), axis = 1)

df['fries'] = df.apply(lambda x: int(x.fries) - (x.big_combo * 1), axis = 1)

df['drink'] = df.apply(lambda x: int(x.drink) - (x.big_combo * 1), axis = 1)

'================================================'

def snack_count(row):
    fries_num = int(row['fries']) // 1 
    drink_num = int(row['drink']) // 1 

    if fries_num > 0 and drink_num > 0:
        return min(fries_num, drink_num)
    else:
        return 0


df['snack_combo'] = df.apply(snack_count, axis = 1)

df['fries'] = df.apply(lambda x: int(x.fries) - (x.snack_combo * 1), axis = 1)

df['drink'] = df.apply(lambda x: int(x.drink) - (x.snack_combo * 1), axis = 1)

df

In [31]:
df

,Unnamed: 0,date,branch,order,burger,fries,drink,family_combo,big_combo,snack_combo
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink",0,0,1,0,1,3
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink",0,1,0,0,2,1
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink",0,2,0,0,2,0
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink",0,5,0,0,0,1
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink",5,0,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink",0,0,3,0,0,3
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries",7,6,0,0,0,0
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries",4,6,0,0,0,0
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink",2,1,0,1,1,0


In [39]:
# 3. Calculate the total order value.
fam_price = pricedf.loc['Fam_combo']['price']
big_price = pricedf.loc['Big_combo']['price']  
snack_price = pricedf.loc['Snack_combo']['price']
burger_price = pricedf.loc['S_Burger']['price']
fries_price = pricedf.loc['S_Fries']['price']
drink_price = pricedf.loc['S_Drink']['price']

#df["sales"] = df["family_combo"]*pricedf["Fam_combo"] + df["big_combo"]*pricedf["Big_combo"] + df["snack_combo"]*pricedf["Snack_combo"] + df["burger"]*pricedf["S_Burger"] + df["fries"]*pricedf["S_Fries"] + df["drink"]*pricedf["S_Drink"]

df['sales'] = df.apply(lambda x: x.burger*burger_price + x.fries*fries_price + x.drink*drink_price + x.family_combo*fam_price + x.big_combo*big_price + x.snack_combo*snack_price, axis = 1)
df

,Unnamed: 0,date,branch,order,burger,fries,drink,family_combo,big_combo,snack_combo,sales
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink",0,0,1,0,1,3,515
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink",0,1,0,0,2,1,506
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink",0,2,0,0,2,0,476
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink",0,5,0,0,0,1,444
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink",5,0,6,0,0,0,739
...,...,...,...,...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink",0,0,3,0,0,3,444
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries",7,6,0,0,0,0,1037
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries",4,6,0,0,0,0,770
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink",2,1,0,1,1,0,1015


In [40]:
# 4. Determine which branch made the most revenue, and how much?
df.groupby('branch').sum(numeric_only = True)['sales'].sort_values(ascending = False)

branch
Branch 4     8761312
Branch 9     8721850
Branch 1     8701036
Branch 6     8687340
Branch 3     8684900
Branch 10    8679712
Branch 7     8653137
Branch 8     8607299
Branch 2     8602327
Branch 5     8554370
Name: sales, dtype: int64